In [1]:
from __future__ import print_function
from six.moves import cPickle as pickle
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import tensorflow as tf

In [2]:
f=open('Data.pickle','rb')
Data=open('Data.pickle','rb')
train_dataset=pickle.load(Data)['train_dataset']
Data.seek(0)
train_labels=pickle.load(Data)['train_labels']
Data.seek(0)
test_dataset=pickle.load(Data)['test_dataset']
Data.seek(0)
test_labels=pickle.load(Data)['test_labels']
Data.close()

In [3]:
count=0
Corrupted_data=np.zeros((28,28),dtype=np.float32)
for i in range(train_dataset.shape[0]):
    if np.array_equal(train_dataset[i,:,:],Corrupted_data):
        Train_data[count,:,:]=train_dataset[i,:,:]
        Train_label[count]=train_labels[i]
        count=count+1
print(count)        
print(train_labels.shape)
print(train_dataset.shape)

train_dataset=train_dataset.reshape([train_dataset.shape[0],train_dataset.shape[1],train_dataset.shape[2],1])
test_dataset=test_dataset.reshape([test_dataset.shape[0],test_dataset.shape[1],test_dataset.shape[2],1])
print(test_dataset.shape)
print(train_dataset.shape)
print(test_labels.shape)

0
(50000,)
(50000, 28, 28)
(900, 28, 28, 1)
(50000, 28, 28, 1)
(900,)


In [4]:
def One_hot_encoder(labels,C):
    C=tf.constant(C,dtype=tf.int32,name='C')
    encoded=tf.one_hot(labels,C)
    with tf.Session() as sess:
        return sess.run(encoded)

In [5]:
Train_labels=One_hot_encoder(train_labels,10)
W1=tf.get_variable('W1',[3,3,1,10],initializer=tf.contrib.layers.xavier_initializer(seed=0))
W2=tf.get_variable('W2',[5,5,10,8],initializer=tf.contrib.layers.xavier_initializer(seed=0))

In [16]:
Z1=tf.nn.conv2d(train_dataset,W1,strides=[1,1,1,1],padding='SAME')
A1=tf.nn.relu(Z1)
P1= tf.nn.max_pool(A1,padding='SAME',ksize=[1,4,4,1],strides=[1,4,4,1])
Z2=tf.nn.conv2d(P1,W2,strides=[1,1,1,1],padding='SAME')
A2=tf.nn.relu(Z2)
P2=tf.nn.max_pool(A2,padding='SAME',ksize=[1,4,4,1],strides=[1,4,4,1])
P2= tf.contrib.layers.flatten(P2)
Z3 = tf.contrib.layers.fully_connected(P2,10,activation_fn=None)
print(Z3.shape)
print(train_labels.shape)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Train_labels))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(cost)
predict_op = tf.argmax(Z3, 1)
correct_prediction = tf.equal(tf.argmax(Z3, 1), tf.argmax(Train_labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
init = tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
for i in range(100):
    sess.run(train_op)
    cost,acc=sess.run([cost,accuracy])
    print(1,":",cost,":",acc)


(50000, 10)
(50000,)
1 : 2.31147 : 0.13202


TypeError: Fetch argument 2.3114748 has invalid type <class 'numpy.float32'>, must be a string or Tensor. (Can not convert a float32 into a Tensor or Operation.)